# Find beta bound

In [1]:
import numpy as np
from scipy.optimize import minimize

For a set of measurements $\mathbf{o}=(o_1,o_2,..,o_k)^T$ in $\mathbb{C}^{d=2j+1}$ we will compute the boud beta, $[\langle\mathbf{o} \rangle_\psi]^T\langle\mathbf{o} \rangle_\psi\leq \beta$ by maximizing numerically over single-atom states $\psi$. Another option is to find an analytical upper bound as

$$||\mathbf{R}_{0}||^2+\kappa\left(1-\frac{1}{d} \right)\max_{\rm eigenvalue}|R^T R|  \geq \beta \ ,$$

where $\mathbf{R}_0=[\mathrm{Tr}[\mathbf{o}]]^T\in \mathbb{R}^k$ and $R=\mathrm{Tr}[\boldsymbol{\lambda}[\boldsymbol{o}]^T] \in \mathbb{R}^k\times \mathbb{R}^{d^2-1}$and $\boldsymbol{\lambda}$ is the Gell-Mann basis of traceless hermitian operators fulfilling the orthogonality relation $\mathrm{Tr}[\boldsymbol{\lambda}\boldsymbol{\lambda}^T]=\kappa\mathbb{I}_{d^2-1}$. In general the upper bound will not be tight and will not be reproduced by a valid quantum state $\psi$, so the numerical exploration is necessary. Also the function to maximize is not concave on the variables, so we can not use concave optimization theory.



In [2]:
def beta_ana(o):
    kappa = np.trace(gellmann[0] @ gellmann[0])
    Rk = np.einsum("kii", o)
    Rkl = np.einsum("kij,lji->kl", o, gellmann)
    return np.real(np.dot(Rk,Rk) + kappa*(1-1/(2*j+1))*np.max(np.abs(np.linalg.eigh(Rkl.T @ Rkl)[0])))


def beta_num(o):
    def beta_x(x): 
      x = x[:2*j+1]+1j*x[2*j+1:]
      x = x/np.linalg.norm(x)
      W = np.einsum("i,tij,j->t",x,o,x.conj()).real
      return -np.real(np.vdot(W,W))
    n_trials = 10
    b = -np.min([minimize(beta_x, np.random.rand(2*(2*j+1))).fun for i in range(n_trials)])
    return b

We define the Gell-Mann, the spin-$j=(d-1)/2$ and the projectors to spin sublevels bases: 

In [3]:
def gm(j):
    d=2*j+1
    def gellmann(j, k, d):  
        if j > k:
            gjkd = np.zeros((d, d), dtype=np.complex128)
            gjkd[j - 1][k - 1] = 1
            gjkd[k - 1][j - 1] = 1
        elif k > j:
            gjkd = np.zeros((d, d), dtype=np.complex128)
            gjkd[j - 1][k - 1] = -1.j
            gjkd[k - 1][j - 1] = 1.j
        elif j == k and j < d:
            gjkd = np.sqrt(2/(j*(j + 1)))*np.diag([1 + 0.j if n <= j
                                                   else (-j + 0.j if n == (j + 1)
                                                         else 0 + 0.j)
                                                   for n in range(1, d + 1)])
        else:
            gjkd = np.diag([1 + 0.j for n in range(1, d + 1)])
        return gjkd
    gm = [gellmann(1+i, 1+j, d) for i in range(d) for j in range(d)]
    return [gm[i] for i in range(d**2-1)]

def spin(j):
    mz = np.arange(-j, j+1)
    sp = np.diag(np.sqrt(j * (j+1) - mz[:-1]*(mz[:-1]+1)), k=-1)
    jx = (sp+sp.T)/2
    jy = -(sp-sp.T)/(2 * 1j)
    jz = -np.diag(mz)
    return [jx,jy,jz]

def n(s,j): # m\in \{-j,-j+1,..,j\} 
    px = np.matrix([np.linalg.eigh(spin(j)[0])[1][:,s+j]])
    py = np.matrix([np.linalg.eigh(spin(j)[1])[1][:,s+j]])
    pz = np.matrix([np.linalg.eigh(spin(j)[2])[1][:,s+j]])
    return [np.array(px.H*px), np.array(py.H*py),np.array(pz.H*pz)]

In th present work we consider two bases: One composed of spin and projection to zero spin sublevel, and the other all the sublevel projections except zero for the three cartesian orthogonal directions $\{x,y,z\}$:

$$\mathcal{B}_1=\{j^{(a)}, n_0^{(a)} \}_{a\in \{x,y,z\}}$$

$$\mathcal{B}_2=\{\{ n_s^{(a)} \}_{s=-j\neq o}^j\}_{a\in \{x,y,z\}}$$

In [4]:
def basis_1(j):
    return spin(j) + n(0,j)

def basis_2(j):
    basis =[]
    for s in [*range(-j, 0), *range(1, j+1)]:
        basis += n(s,j)
    return basis

As an example, for $j=1$ we obtain the bounds:

In [5]:
j = 1
gellmann = gm(1)

In [6]:
beta_num(basis_1(j))

1.4999999999999347

In [7]:
beta_ana(basis_1(j))

8.333333333333336

In [8]:
beta_num(basis_2(j))

1.2499999999995932

In [9]:
beta_ana(basis_2(j))

8.66666666666667